# Hyperflow 2023: **Python for FCM (forget R 😉)**

This notebook gives a brief introduction of processing Flow Cytometry (FCM) data with python. Most of the featured examples are based on the open-source Python framework [Cytoflow](https://cytoflow.readthedocs.io/en/stable/). Please check it out for further information.

First we import matplotlib (a plotting framework for python) and cytoflow. We configure some default settings to enable larger plots in this notebook.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
%matplotlib inline
    
import cytoflow as flow
# flow.DefaultKWArgs["height"] = 4

#### Reading FCS files

Next we read in FCM measurements. We can either read  data from a FCS-file or from a CSV-file.

In [2]:
basepath = "./data/"
tube1 = flow.Tube(file = basepath + '494.csv')
ex = flow.ImportOp(tubes = [tube1]).apply()

Everything in Cytoflow is build around experiments. An experiment is the primary object that holds FCM measurements and metadata of several tubes. Core structure within the experiment is a Pandas Dataframe:

In [3]:
ex.data

,TIME,FSC-A,FSC-W,SSC-A,CD20,CD10,CD45,CD34,SY41,CD19,CD38,APC-CY7-A
0,0.256109,0.724819,0.000000,0.993575,0.511647,0.737258,3.116999,0.610369,2.394137,2.838983,1.824015,1.053927
1,0.256109,2.111998,1.095594,0.789835,2.883300,4.009430,3.395909,2.123596,3.296094,2.423301,1.525653,1.349321
2,0.256109,2.071740,1.094057,4.499983,1.406279,1.910993,2.866903,0.984716,3.535055,1.830034,2.072034,0.882448
3,0.256150,1.923883,1.095217,0.648713,1.031393,4.020207,2.840137,0.942146,3.296256,2.922170,1.724284,0.740326
4,0.256190,1.580198,1.005982,0.781159,1.264268,1.307244,3.775684,0.514218,2.890679,1.018567,1.652686,0.857172
...,...,...,...,...,...,...,...,...,...,...,...,...
83406,4.499838,4.229612,1.110860,4.499983,1.581865,1.840961,2.920752,1.529190,3.640661,1.976877,2.090316,0.663859
83407,4.499919,1.721509,1.086372,0.589498,2.136454,3.867376,3.045547,2.038175,2.996041,1.974872,2.348283,0.862303
83408,4.499959,1.888425,1.035353,1.412427,3.039960,4.131422,3.501659,2.281008,3.293125,3.573528,3.040444,1.108402
83409,4.500000,1.479483,1.069826,0.567807,1.333666,4.141739,3.046796,1.852519,3.120067,2.561236,1.447637,0.767203


Pandas Dataframes are one of the most commonly used structures when manipulating tabular data in Python. We can do all sort of things with it like selecting, transforming and cleaning data instances.

## Views in Cytoflow
Let's plot our experiment with Cytoflow. We first define a View where we specify which channels we want to visualize. And then we use this view to plot our previously created experiment. Cytoflow always follows this pattern: First we define the parameters of our view and then we plot it with a particular experiment.

In [ ]:
scatter_cd19cd45 = flow.ScatterplotView(xchannel = 'CD19', ychannel = 'CD45')
scatter_cd19cd45.plot(ex)

There are many different views in Cytoflow such as Histograms, Bar Charts, Violin Plots or Kernel Density Estimates.

In [ ]:
kde_cd19cd45 = flow.Kde2DView(xchannel = 'CD19', ychannel = 'CD45') 
kde_cd19cd45.plot(ex)

## Operations in Cytoflow
Besides *Views* there are exists *Operations* that allows us to manipulate experiments in Cytoflow. Consider the ThresholdOperation:

In [ ]:
cd19_thresh = flow.ThresholdOp(name = "CD19pos",channel = 'CD19', threshold = 1.5)

ex2 = cd19_thresh.apply(ex)



We first defined the Operation and then applied to our experiment. This results in a new experiments, which now contains a condition that indicates which events are above the threshold which are below.

In [ ]:
ex2.conditions


Operations in Cytoflow also often have a default view that visualizes the Operation applied to an experiment.

In [ ]:
cd19_thresh.default_view().plot(ex2)

We can now also use the CD19pos condition to add color to our scatter plot:

In [ ]:
scatter_cd19cd45.huefacet = 'CD19pos'
scatter_cd19cd45.plot(ex2)

We can do the same with our KDE plot.

In [ ]:
kde_cd19cd45.huefacet = 'CD19pos'
kde_cd19cd45.plot(ex2)

We can also load predefined conditions from a file. Like here we load conditions from a csv files that present that result of manuell gating applied to our sample.

In [ ]:
human_conditions = flow.BulkConditionOp(conditions_csv_path =  basepath + '494_labels.csv',
                                      combine_order = ["syto", "singlets", "intact","cd19", "blast"],
                                      combined_conditions_name="human_gt",
                                      combined_condition_default ="other")

ex3 = human_conditions.apply(ex2)

Each column in this CSV-File specifies if a particular event is inside the given gate. We define a combine order which essientially defines the gating hierachy. With this we end up with a column that shows the name of the last gate a event is into.
We can see the difference when we now observe our dataframe. We have a new column foreach gate as we as a combined column showing the "cell type".

In [ ]:
ex3.data

We now can use this human generated ground truth to subsample our FCM data in a useful way. We specify with the SubsampleOp that we want 1000 events of each label that is present in human_gt.

In [ ]:
subsample_op = flow.SubsampleOp(sampling_type = "absolute", sampling_def = {"human_gt":"1000:FromEach"})

ex4 = subsample_op.apply(ex3)

As you can see now we only have exactly 6k events.

In [ ]:
ex4.data

## Advanced Operations
Let's do some dimensionality reduction with UMAP (UMAP is like t-SNE, only better).

In [ ]:
ex4.channels

Let's apply umap but only to the channels that are actually useful. So we won't use APC-CY7 and TIME.

In [ ]:
umap = flow.UMAPOp(name= "umap", channels = ex4.channels[1:-1])
umap.estimate(ex4)

ex5 = umap.apply(ex4)

UMAP is an operation that needs to be estimated before applied to any experiment. In this estimation step the dimensionality reduction is calculated. We can see that we now have two addtional channels: umap_1 and umap2.

In [ ]:
ex5.channels

Like the other operations also UMAP has a default view.

In [ ]:
umap.default_view().plot(ex5)

We see that UMAP clustered our events but we don't really know if those clusters make sense. Lets color the events by human ground truth.

In [ ]:
umap.default_view(huefacet="human_gt").plot(ex5)

Now we can see that those clusters reflect quite well the cell population found by human operators. We can also run a clustering algorithm on the embedded data. Here we use HDBSCAN.

In [ ]:
clustering = flow.HDBSCANOp(name = "hdbscan", min_cluster_size  = 25, channels= ["umap_1", "umap_2"])
clustering.estimate(ex5)

ex6 = clustering.apply(ex5)

In [ ]:
umap.default_view(huefacet="hdbscan").plot(ex6)

## Machine Learning

Python is especially powerful when it comes to the number of machine learning packages. In the following we load a Deep Learning model from the popular platform Hugging Face. [flowformer](https://huggingface.co/matth/flowformer) is model trained on B-ALL FCM data for detection of blast cells.

In [ ]:
from transformers import AutoModel
flowformer = AutoModel.from_pretrained("matth/flowformer", trust_remote_code=True)


Now we need to prepare our FCM data to be a suitable input for the model. First we select which data channels we want to pass to the model.

In [ ]:
ex6.channels

We don't need to umap channels therefore we pass all the channels except the last two.

In [ ]:
markers_to_use = ex6.channels[1:-2]

And we also need to convert our data into a Tensor (the datatype of [Pytorch](https://pytorch.org/)).

In [ ]:
import torch
events_tensor = torch.Tensor(ex6.data[markers_to_use].to_numpy()).unsqueeze(dim=0)


In [ ]:
len(markers_to_use)

Now we can simply pass our events for the prediction.

In [ ]:
output = flowformer(events_tensor, markers = markers_to_use)
output

'prediction' tells us if the model predicted our events as blasts (1) or non-blasts (0).

In [ ]:
output["prediction"] = output["prediction"].squeeze().numpy()

In [ ]:
output["prediction"]

The result can be plotted in our UMAP by adding the prediction as condition.

In [ ]:
ex6.add_condition("blast_pred",dtype="bool", data =output["prediction"])


In [ ]:
umap.default_view(huefacet = "blast_pred").plot(ex6)

In the UMAP plot we can see that the model predicted the this one cluster as blast cluster. But we can also get more percise performance results using a nother common Python Machine Learning toolkit called sklearn.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ex6.data["blast"], ex6["blast_pred"]))